The Turán number of $C_5^-$
==============================

This notebook contains calculations for the Turán number of $C_5^-$

As the blowup of $K_4^-$ contains $C_5^-$, we can additionally assume 
that we work in $K_4^-$-free structures. 

1. The first cell sets up the combinatorial theory of $C_5^-$ and 
$K_4^-$-free 3-graphs (called TGp). In addition, it sets up the 
combinatorial theory on the same 3-graphs with vertices colored 
from 3 possible colors (called CTGp)

2. The second cell performs the basic calculation of upper bounding
edges in the theory. It gives the 1/4 + 1/1000 (in fact 0.250728)
upper bound.

3. That upper bound is used in the following cell. It lower bounds
k221-2*k311 and k222/k111 at the optimum.

4. There is a quick sanity check below that, to check the coefficients
of k221, k311, f222, k222

5. The last cell works in the colored theory, and shows that the bad
edges have number smaller than the missing good edges. It uses the upper
bound about k222/k111.

In [1]:
###
### This cell is just to set up the theory.
### In practice it is not needed, as the calculations (multiplication table and generated structures)
### are already done and saved. But it is here for completeness (and for re-runs from scratch)
###

from sage.algebras.flag_algebras import *

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    """
    Helper function to check is any of the smalls appears in each of the larges.

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    larges - list of flags, also must be from a theory with edges relation

    OUTPUT:
    list of booleans, i-th element represents if i-th large flag is free from all smalls
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    """
    Helper function to check is any of the smalls appears in the large, and if yes,
    returns an injection of the small

    INPUT:
    smalls - list of flags, must be from a theory with edges relation
    large - a flag, also must be from a theory with edges relation

    OUTPUT:
    empty list ([]) if all smalls is avoided, otherwise [small, mapping] telling how to
    inject small into large with mapping
    """
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
C5m = TG(5, edges=[[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 0]])

# flags of size 5
fl5 = TG.generate_flags(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([C5m], fl5)

# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl5 = TG.generate_flags(5)
fl6 = TG.generate_flags(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("No5Cm", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
# should be 9 55 1127
print(len(TGp.generate_flags(5)), len(TGp.generate_flags(6)), len(TGp.generate_flags(7)))






# This is code to create colored theories
# This is a default code for all color partition
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    g_parts = [[ii] for ii in ftype_points] + \
              [[ii for ii in range(n) if ii not in ftype_points]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

# This is also a default code for all color partition
def _generateCT(base_theory, k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    BT = base_theory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

# To make the default codes work for this specific case:
# The generator:
# Colors the elements of TGp (3-graphs without C5- and K4-), works on 3-uniform structures
# and the colors 0, 1, 2 are interchangeable (otherwise it would say [[0], [1], [2]]
def generate_colored(n):
    return _generateCT(TGp, 3, [[0, 1, 2]], n)

# Same for the identifier. Colors are interchangeable.
def identify_colored(n, ftype_points, edges, C0, C1, C2):
    return _identifyCT(3, [[0, 1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

# CTGp is the colored variant of TGp (NoC5m)
CTGp = CombinatorialTheory("ColoredNo5Cm", generate_colored, identify_colored, edges=3, C0=1, C1=1, C2=1)

# sanity check, the number of flags with size 4, 5, 6
# should be 18 132 2840
print(len(CTGp.generate_flags(4)), len(CTGp.generate_flags(5)), len(CTGp.generate_flags(6)))

9 55 1127
18 132 2840


In [10]:
###
### This is where the actual calculation starts.
### This part just gives a standard upper bound on the number of edges
###

# degree equality 
p2f4 = TGp.generate_flags(4, TGp(2, ftype=[0, 1]))
degree_difference = p2f4[2]-p2f4[3]+p2f4[5]-p2f4[6]

standard_assums = [degree_difference, -degree_difference]

#run the optimizer
max_edge = TGp.optimize_problem(TGp(3, edges=[[0, 1, 2]]), 7, maximize=True, positives=standard_assums)

#should be around  0.25072863788449695 < 1/4 + 1/1000
print("\n\n", max_edge, "\n\n")

Ftypes constructed in 64.57s
Block sizes done in 1.18s
Block sizes are [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28, -1127, -192]
Calculating product matrices for 12 ftypes and 1127 structures
Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] is complete: : 12it [00:03,  3.45it/s]
Table calculation done in 3.49s
Target and constraint calculation done in 0.22s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.25072863788449695
Iter:  1 Ap: 4.78e-01 Pobj: -5.9027977e+01 Ad: 2.34e-01 Dobj:  2.5209567e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -1.3500045e+02 Ad: 7.59e-01 Dobj:  3.0296639e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -1.4671819e+02 Ad: 7.86e-01 Dobj:  2.2062351e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.4535501e+02 Ad: 9.34e-01 Dobj: -1.5262505e-01 
Iter:  5 Ap: 9.05e-01 Pobj: -1.4433981e+02 Ad: 8.98e-01 Dobj: -1.8951766e-01 
Iter:  6 Ap: 9.06e-01 Pobj: -1.8608619e+02 Ad: 4.51e-01 Dobj: -1.9250127e-01 
Iter:  7 Ap: 5.52e-01 

In [2]:
###
### This code block deals with the uncolored part of the calculation (minimizing k221-2*k311 and k222/k111)
###




import itertools
# base edges for f222 (0, 1, 2) is the center and (3, 4, 5) is the good edge
be = [[0, 1, 2], [3, 4, 5], [0, 1, 5], [0, 2, 4], [1, 2, 3]]
# additional edges for f222. These are the edges we can include while still not violating the no C5m condition.
ae = [[0, 4, 5], [1, 3, 5], [2, 3, 4]]

# f221 and f311 just a sum of two elements, so can write it explicitely. They are typed, so no automorphism consideration is needed
f221 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [0, 2, 4], [2, 3, 4]])
f311 = TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4]]) + TGp(5, ftype=[0, 1, 2], edges=[[0, 1, 2], [1, 2, 3], [1, 2, 4], [0, 3, 4]])

# f222, it is a sum of 8 flags, so simply use this iterator to loop through all subsets of the additional edges
f222 = TGp(6, ftype=[0, 1, 2], edges=be)
for ii in [1, 2, 3]:
    for xx in itertools.combinations(ae, int(ii)):
        f222 = f222 + TGp(6, ftype=[0, 1, 2], edges=be+list(xx))

# these are the projected values. Here the automorphism groups are taken care of.
k221 = f221.project()
k311 = f311.project()
k222 = f222.project()

#standard positivity constraints.

# edge density bounds, the edge <= 1/4 + 1/1000 is used here from the previous cell
edge_lower = TGp(3, edges=[[0, 1, 2]]) - 1/4
edge_upper = 1/4 + 1/1000 - TGp(3, edges=[[0, 1, 2]])

# alpha for some of the calculations
alpha = 1-(1/100)

# the list of positivity assumptions
uncolored_assums = [edge_lower, edge_upper, degree_difference, -degree_difference]

# This is for minimizing f222, which is a flag, so technically it is the quotient k222 / k111
min_f222 = TGp.optimize_problem(f222, 7, maximize=False, positives=uncolored_assums)

# This is for minimizing k221 - (2*alpha)*k311
# On the optimal construction k221 = 2*k311, so alpha guarantees this is slightly positive
min_k2m3 = TGp.optimize_problem(k221 - (2*alpha)*k311, 7, maximize=False, positives=uncolored_assums)

#they should be around 0.19495185630466152 and 0.0003212179234714776
#sometimes the print statements get tangled in the output of the CSDP. But the variables are set,
#they can be checked in a different cell
print("\n\n", min_f222, "\n\n", min_k2m3, "\n\n")

Ftypes constructed in 65.71s
Block sizes done in 0.08s
Block sizes are [5, 74, 32, 388, 178, 83, 97, 34, 45, 54, 31, 28, -1127, -204]
Calculating product matrices for 12 ftypes and 1127 structures
Ftype on 5 points with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] is complete: : 12it [00:03,  3.57it/s]
Table calculation done in 3.37s
Target and constraint calculation done in 0.79s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 0.19495185630466152
Iter:  1 Ap: 1.73e-01 Pobj: -8.3448809e+01 Ad: 6.51e-02 Dobj:  5.3454227e-01 
Iter:  2 Ap: 8.03e-01 Pobj: -4.2124663e+02 Ad: 2.77e-01 Dobj:  2.2698892e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -5.7083161e+02 Ad: 8.39e-01 Dobj:  5.0819956e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.8089785e+02 Ad: 8.78e-01 Dobj:  1.1975895e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -5.7886145e+02 Ad: 8.51e-01 Dobj:  6.3523286e-02 
Iter:  6 Ap: 1.00e+00 Pobj: -7.5226615e+02 Ad: 4.81e-01 Dobj:  5.8723223e-02 
Iter:  7 Ap: 2.40e-01 

In [4]:
# to check the coefficients. These sanity checks are easy to do in general
print("k221 is: ", k221, "\n\nk311 is: ", k311, "\n\nf222 is: ", f222, "\n\nk222 is: ", k222)

k221 is:  Flag Algebra Element over Rational Field
0    - Flag on 5 points, ftype from [] with edges=[]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3], [0, 1, 4]]
1/30 - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [0, 1, 3]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3]]
2/15 - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [0, 1, 3], [0, 2, 4]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 3, 4], [1, 2, 3], [1, 2, 4]] 

k311 is:  Flag Algebra Element over Rational Field
0    - Flag on 5 points, ftype from [] with edges=[]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2]]
0    - Flag on 5 points, ftype from [] with edges=[[0, 1, 2], [0, 1, 3]]
0    

In [6]:
###
### This is the code that performs the calculations on the colored theory
###


#edge with correct colors
C = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]])
#pointed edge with correct colors
Cp = CTGp(3, edges=[[0, 1, 2]], C0=[[0]], C1=[[1]], C2=[[2]], ftype=[0])

#edge with bad colors (since color-blind, this includes the bad edges looking the wrong way too)
B = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [1]], C1=[[2]])

#pointed edge with bad colors
Bp = CTGp(3, edges=[[0, 1, 2]], C0=[[0], [2]], C1=[[1]], C2=[], ftype=[0])

#missing edge with good colors
M = CTGp(3, edges=[], C0=[[0]], C1=[[1]], C2=[[2]])

#positivity assumptions
#each point, good edges are more than bad edges (divided by two due to the wrong color order)
#edge density is larger than 0.194 (from previous calculation)
colored_assums = [Cp - Bp/2, C - 194/1000]

#bad is less than missing
max_bmm = CTGp.optimize_problem(B + (-1)*M, 6, maximize=True, positives=colored_assums)
#should be around 1.5177109841127238e-08
print("\n\n", max_bmm, "\n\n")

Ftypes constructed in 28.59s
Block sizes done in 7.32s
Block sizes are [35, 57, 82, 123, 123, 123, 40, 60, 60, 60, 60, 60, 20, 30, 30, 30, 30, 30, 30, 30, -2840, -100]
Calculating product matrices for 20 ftypes and 2840 structures
Ftype on 4 points with edges=[[0, 1, 2], [0, 1, 3]], C0=[[0]], C1=[[1]], C2=[[2], [3]] is complete: : 20it [00:03,  6.45it/s]    
Table calculation done in 3.10s
Target and constraint calculation done in 1.64s

CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Result is 1.5177109841127238e-08
Iter:  1 Ap: 1.82e-01 Pobj: -1.4813544e+01 Ad: 1.63e-01 Dobj:  7.0264950e+01 
Iter:  2 Ap: 4.13e-01 Pobj: -5.0048121e+01 Ad: 5.75e-01 Dobj:  3.3776817e+01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.2345465e+02 Ad: 4.59e-01 Dobj:  1.8780183e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.3364263e+02 Ad: 8.86e-01 Dobj:  2.4999848e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -1.4041485e+02 Ad: 9.23e-01 Dobj:  2.0577974e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.5110402e+02 Ad: